# 卷积神经网络(CNN)

卷积神经网络(CNN), 是一种主要用于处理图像的深度神经网络. 其网络结构主要包括**卷积(Conv)操作**与**池化(Pool)操作**, 其中卷积操作可能涉及到**filter卷积核的学习**, **stride步长**与**padding填充**的选择. 主要可以完成如下任务:

- **图片分类(Image Classification)**: 分辨图片内容属于哪一类, 数据集常使用MINIST, CIFAR, ImageNet等.
- **物体定位(Object Localization)**: 预测包含目标物体的区域, 数据集常使用ImageNet等.
- **物体识别(Object Recognition)**: 分类并定位图片中出现的所有物体. 数据集常用PASCAL, COCO等.
- **语义分割(Semantic Segmentation)**: 将图像中的每一个像素分到其所属的物体类别. 数据集常用PASCAL, COCO等.
- **实例分割(Instance Segmentation)**: 将图像中的每一个像素分到其所属物体实例。数据集：PASCAL, COCO等.

处理图片最大的问题在于, 图片像素累加起来的维度非常高, 假如是$1000\times1000$的图片, 其输入特征维度高达$1000\times1000\times3=3 billion$.

## 背景: 边缘检测

检测一个完整的物体可能经过包括: 边缘检测, 部分区域检测, 完整物体检测等几个步骤. 其中边缘检测是检测的第一步, 它会用到**卷积操作**, 卷积操作会使用一个称为**过滤器(卷积核)** 的矩阵 (下图filter), 卷积操作如下图所示:

![cnn1](./resources/cnn1.png)

上图左边为一张$6*6$的黑白图片,中间为卷积核,右边为经过卷积操作得到的特征map.

**为什么上面的卷积操作可以检测到边缘**: 可以这样想, 假设处于该图像像素分布均匀的区域,经过卷积核的卷积,-1区的乘积与1区的乘积都抵消了,这样这些区域卷积后的值都等与0. 只有那些左边右边的像素值有差异的区域可以得到非零的卷积值, 即需要检测到的边缘.

常用的过滤器还有如:Sobel过滤器, Scharr过滤器等,但是经过验证,如果真的想要检测出复杂的图像边缘, **可以把过滤器设置为可训练的参数, 最后通过反向传播来训练**.

## Padding填充

按照上面的卷积操作, 如果我们有一个$n\times n$的图片, 用$f\times\ f$的过滤器卷积,那么输出维度为:

> $(n-f+1)\times(n-f+1)$

这样出现了两个缺点: (1). **每次做卷积,图像都要缩小**; (2). **过滤器到达图像边缘像素的次数相比于图像中心区域更少, 这意味着丢失了很多图像边缘信息**;

通过在图像周围填充一圈或几圈像素可以解决上面的问题, 这种方法称为Padding. 其填充过程可由下图所示:

![cnn2](./resources/cnn2.png)

设$p$为在周围填充的像素数目，上面填充了一个像素点，即$p=1$, 则卷积后的尺寸计算方式如下:

> $(n+2p-f+1)\times(n+2p-f+1)$

填充方式一般有两种: 

- **Valid卷积**: 即不做填充处理, $p=0$;
- **Same卷积**: 要填充, 且填充后经过卷积运算,图片的尺寸不发生变化,这个时候$p=(f-1)/2$;

一般$f$都使用奇数，其原因有两个：(1) **如果使用偶数，会出现不对称填充**；(2) **奇数有中心点，便于指出过滤器位置**；

## 卷积步长（Strided convolutions）

**卷积中的步长即卷积核每次移动的单元数**。如下是长宽步长都为2的例子：

![cnn3](./resources/cnn3.png)

这样假设使用的步长为$s$，卷积后得到的图像大小为：

> $(\dfrac{n+2p-f}{s}+1) \times (\dfrac{n+2p-f}{s}+1)$

当除以$s$后不是一个整数会使用**向下取整**。

以上的整个卷积操作在数学上被称为**互相关**，数学上真正的卷积还有一个把卷积核进行镜像翻转的过程, 但在机器学习领域上面的步骤即被称为卷积操作。

## 三维卷积（Convolutions over volumes）

对于3通道的图片，卷积核的维度必须也是3维的，分别对应红绿蓝3通道，每个通道与对应通道的卷积核相乘，最后**将所有通道的每个值累加起来得到最终特征值**。其操作如下所示：

![cnn4](./resources/cnn4.png)

比如上图最右蓝色点位置的值就是卷积核和图像对应位置乘积得到的27数相加后得到。然后将卷积核立方体向右向下滑动得到其他位置的特征值。

**可以使用多个过滤器以得到不同的特征map**，最终可将得到的特征再重叠起来传入网络的下一层。其操作流程如下：

![cnn5](./resources/cnn5.png)

## 单层卷积神经网络

单层卷积神经网络在上面三维卷积的基础上增加了两个操作：

- **对产生的特征map添加bias项（每个特征map里的特征值都加一个相同的bias）**;
- **对添加完bias项后的每个特征值再套用一个激活函数，如Relu**；

其操作过程如下所示：

![cnn6](./resources/cnn6.png)

如果将上面的两个filter写成$W^{[1]}$，偏置单元为$b^{[1]}$，则：

> $z^{[1]}=W^{[1]}a^{[0]}+b^{[1]}$, 其中$a^{[0]}=X$,  
> $a^{[1]}=g(z^{[1]})$, 这里$g$是激活函数

上面一层卷积的总参数个数为$27*2+2=56$, **这相比于全连接参数大大减少了**。下面定量的描述下CNN中一层的参数变化：

使用$n_H^{[l-1]}*n_W^{[l-1]}*n_c^{[l-1]}$表示第$l-1$层的输出，即图片的高度\*宽度\*通道数; 用$f^{[l]}$表示第$l$层过滤器的大小，其维度为：$f^{[l]}*f^{[l]}*n_c^{[l-1]}$; 用$p^{[l]}$表示第$l$层padding的大小;用$s^{[l]}$表示第$l$层步长大小。这样第$l$层的输出为：

> $n_H^{[l]} = \dfrac{n_H^{[l-1]}+2p^{[l]}-f^{[l]}}{s^{[l]}}+1$  
> $n_W^{[l]} = \dfrac{n_W^{[l-1]}+2p^{[l]}-f^{[l]}}{s^{[l]}}+1$  
> $n_c^{[l]}=number \ of \ filters$

按照常见的多层卷积神经网络的结构设计, 一般会出现的情况为：**图片大小逐渐减小，通道数目逐渐增大**。

典型的CNN一般包括3层：**卷积层(Conv)**, **池化层(Pool)**，**全连接层(FC)**。

CNN要训练的参数包括: 各层卷积核+卷积核偏置.

## 池化层（Pooling layers）

池化层可以缩减模型的大小，提高计算速度，同时提高所提取特征的鲁棒性。一般常使用最大池化操作，操作方式如下：

![cnn7](./resources/cnn7.png)

池化的超参数包括**过滤器大小$f$**和**步幅$s$** ，上图相当于使用了一个filter尺寸为$2*2$，步长为2的取最大值的过滤器。需要注意的**池化层没有超参数需要学习**。另外还有一个平均池化但用的不多。

在池化操作中，一般都使用$f=2, s=2$的超参数，这相当于高度宽度都缩减一般。

一般把卷积层与池化层当做一层，因为卷积层没有可训练的参数。

## 卷积神经网络示例

一个完整的CNN示例如下，假设要做Mnist手写体识别：

![cnn8](./resources/cnn8.png)

上图中经过了两个卷积池化层(ConvPool),再经过两个全连接层(FC),最后通过一个softmax层完成多分类.

相比于只使用全连接层, 卷积层的两个主要优势在于：**参数共享**与**稀疏连接**。

- 所谓参数共享在于整张图片都可以使用一个卷积核来提取特征,而不是每个区域需要一个独特的卷积核;
- 稀疏连接在于特征map上的每一个特征都只通过原始图片上一部分经过卷积操作得到,跟图片上其他区域无关;

## 参考文献

- [第四门课 卷积神经网络（Convolutional Neural Networks）](http://www.ai-start.com/dl2017/html/lesson4-week1.html)